# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

## AN ENTREPRENEURIAL SEARCH TO OPEN A CHAIN OF INDIAN RESTAURANTS IN NEW YORK

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

# Matplotlib and associated plotting modules

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported successfully')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported successfully


### Load and Explore New York city geographical coordinates dataset

The objective is to find the best locations for opening the new Indian restaurant:

•	 so we are going to analyze the dataset corresponding to New York Neighborhoods. New York comprises of a total of 5 boroughs and 306 neighborhoods. In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the representative neighborhoods that exist in each borough as well as the latitude and longitude coordinates.

This dataset exists for free at the NYU Spatial Data Repository. Link to the dataset is:
https://geo.nyu.edu/catalog/nyu_2451_34572


The json file is downloaded and it is placed on the server. So run a wget command and access the data.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


### Features of the dataset

In [3]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

### Extracting the data into the pandas dataframe

In [4]:

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude



### Looping through the data and filling the dataframe one row at a time:

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Making sure that the dataframe has all 5 boroughs

In [8]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Finding the coordinates of the New York City

In [41]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### The map of New York with neighborhoods superimposed on top

In [110]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [111]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Define Foursquare Credentials and Version

In [112]:
CLIENT_ID = 'EMFY41ZXZRUAEW0BFA10DOQMVOSZ1IT3EWDEXJJHPHML3UEP' # Foursquare ID
CLIENT_SECRET = 'BNREICKTVBXXO2IYIUIJNTYFYFOBUXTT5EXO1EDQXSY4V2B2' #  Foursquare Secret
VERSION = '20190627' # Foursquare API version
LIMIT = 500 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EMFY41ZXZRUAEW0BFA10DOQMVOSZ1IT3EWDEXJJHPHML3UEP
CLIENT_SECRET:BNREICKTVBXXO2IYIUIJNTYFYFOBUXTT5EXO1EDQXSY4V2B2


In [113]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [114]:
#https://developer.foursquare.com/docs/resources/categories
#Indian = 4bf58dd8d48988d10f941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_indian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10f941735')
newyork_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Riverdale Indian Cuisine,40.880886,-73.908800,Indian Restaurant
1,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,40.879331,-73.903192,Indian Restaurant
2,Marble Hill,40.876551,-73.910660,Cumin Indian Cuisine,40.886459,-73.909816,Indian Restaurant
3,Chinatown,40.715618,-73.994279,Kabab Bites,40.720094,-73.995819,Indian Restaurant
4,Chinatown,40.715618,-73.994279,Hampton Chutney Co.,40.720220,-73.999347,Dosa Place


In [115]:
newyork_venues_indian.shape

(1088, 7)

In [116]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [117]:
map_newyork_indian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_indian, 'red', map_newyork_indian)

map_newyork_indian

In [118]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [119]:
manhattan_grouped = newyork_venues_indian.groupby('Neighborhood').count()
manhattan_grouped
print('There are {} uniques categories.'.format(len(newyork_venues_indian['Venue Category'].unique())))

There are 16 uniques categories.


## Analyze each neighborhood

In [120]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_indian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_indian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Himalayan Restaurant,Hookah Bar,Indian Chinese Restaurant,Indian Restaurant,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Chinatown,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [121]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Himalayan Restaurant,Hookah Bar,Indian Chinese Restaurant,Indian Restaurant,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.840000,0.000000,0.00,0.000000,0.040000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.923077,0.076923,0.00,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.058824,0.764706,0.000000,0.00,0.000000,0.000000,0.058824
4,Chinatown,0.000000,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.880000,0.040000,0.00,0.000000,0.000000,0.000000
5,Civic Center,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,0.800000,0.000000,0.00,0.000000,0.033333,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.113636,0.000000,0.000000,0.000000,0.840909,0.000000,0.00,0.022727,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.021277,0.042553,0.000000,0.042553,0.021277,0.000000,0.000000,0.000000,0.829787,0.021277,0.00,0.000000,0.000000,0.021277
9,Financial District,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.769231,0.000000,0.00,0.000000,0.038462,0.000000


In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [123]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
1,Carnegie Hill,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
2,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
3,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Hookah Bar,Himalayan Restaurant
4,Chinatown,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar


## Clustering Neighbours

In [124]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 2, 0, 2, 2, 1, 4, 2], dtype=int32)

In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
1,Manhattan,Chinatown,40.715618,-73.994279,0.0,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck


In [126]:
manhattan_merged.dropna()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
1,Manhattan,Chinatown,40.715618,-73.994279,0.0,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
5,Manhattan,Manhattanville,40.816934,-73.957385,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
6,Manhattan,Central Harlem,40.815976,-73.943211,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
7,Manhattan,East Harlem,40.792249,-73.944182,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
8,Manhattan,Upper East Side,40.775639,-73.960508,0.0,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
9,Manhattan,Yorkville,40.775930,-73.947118,0.0,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
10,Manhattan,Lenox Hill,40.768113,-73.958860,0.0,Indian Restaurant,North Indian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant


In [133]:
manhattan_merged.shape

(40, 15)

In [ ]:


# df contained NaN
manhattan_merged= manhattan_merged[~manhattan_merged['Cluster Labels'].isnull()]

# final conversion to int 
manhattan_merged[['Cluster Labels']] = manhattan_merged[['Cluster Labels']].astype(int)

## Visualizing clusters

In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Cluster 0 : Saturated Market

In [137]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
8,Upper East Side,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
9,Yorkville,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
10,Lenox Hill,Indian Restaurant,North Indian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
11,Roosevelt Island,Indian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
12,Upper West Side,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
13,Lincoln Square,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
18,Greenwich Village,Indian Restaurant,North Indian Restaurant,Food Truck,Dosa Place,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
20,Lower East Side,Indian Restaurant,Vegetarian / Vegan Restaurant,North Indian Restaurant,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
23,Soho,Indian Restaurant,Dosa Place,North Indian Restaurant,Food Truck,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant



# Cluster 1 : Moderately competetive Market

In [65]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
2,Washington Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
4,Hamilton Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
5,Manhattanville,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
6,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
7,East Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
25,Manhattan Valley,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Food Truck
26,Morningside Heights,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Food Truck



# Cluster 2 : Moderately competetive Market

In [66]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
17,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Hookah Bar,Himalayan Restaurant
28,Battery Park City,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
29,Financial District,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
32,Civic Center,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
39,Hudson Yards,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar



# Cluster 3 : Untapped Market

In [67]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Indian Restaurant,Vegetarian / Vegan Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant



# Cluster 4 : Moderately competetive Market

In [68]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Indian Restaurant,Food Truck,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Indian Chinese Restaurant,Hookah Bar
16,Murray Hill,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,North Indian Restaurant,Diner,Tapas Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
19,East Village,Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,North Indian Restaurant,Food Truck,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
21,Tribeca,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
22,Little Italy,Indian Restaurant,Dosa Place,North Indian Restaurant,Food Truck,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
27,Gramercy,Indian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,North Indian Restaurant,Hookah Bar,Food Truck,Deli / Bodega,Chaat Place,Burrito Place,Tapas Restaurant
31,Noho,Indian Restaurant,Dosa Place,Vegetarian / Vegan Restaurant,North Indian Restaurant,Food Truck,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
33,Midtown South,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Diner,Burrito Place,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
38,Flatiron,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,North Indian Restaurant,Hookah Bar,Diner,Deli / Bodega,Burrito Place,Tapas Restaurant
